# Additional Fewshot Examples

This is the code used to benchmark additional fewshot examples.

In [1]:
from openai import OpenAI
from dotenv import load_dotenv
from mistralai import Mistral
load_dotenv()
import pandas as pd
import time
import openai
from tqdm import tqdm
import numpy as np
import csv
import os
client = Mistral(
	api_key=os.getenv('MISTRAL_API_KEY')
)
client_gpt = OpenAI(
	api_key=os.getenv('OPENAI_API_KEY')
)



In [2]:
df = pd.read_csv('./prompts_fewshot.tsv', sep="\t")
df.head()

,Unnamed: 0,class,target,id,system_prompt,user_prompt,function_name
0,0,implicit_hate,['immigrants'],0,Your goal is to classify text as either 'hatef...,glad not to live in ca. taxpayers pay over 13%...,few_shot_basic_random
1,1,implicit_hate,['immigrants'],0,Your goal is to classify text as either 'hatef...,glad not to live in ca. taxpayers pay over 13%...,few_shot_basic_seed
2,2,implicit_hate,['immigrants'],1,Your goal is to classify text as either 'hatef...,"send all illegals away. no need to """"learn mor...",few_shot_basic_random
3,3,implicit_hate,['immigrants'],1,Your goal is to classify text as either 'hatef...,"send all illegals away. no need to """"learn mor...",few_shot_basic_seed
4,4,not_hate,NaN,2,Your goal is to classify text as either 'hatef...,it is not racist and you are not a hate group ...,few_shot_basic_random


In [3]:
df.shape

(3000, 7)

In [4]:
def classify_text(system_prompt, user_prompt, model, max_retries=5, temperature=0.01):
    retries = 0
    while retries < max_retries:
        try:
            completion = client.chat.complete(
                model=model,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=temperature,
                response_format={ "type": "json_object" }
            )
            content = completion.choices[0].message.content
            return content
        except Exception as e:
            retries += 1
            wait_time = 2 ** retries
            print(f"Rate limit hit. Retrying in {wait_time} seconds...")
            time.sleep(wait_time)
    print("Max retries exceeded.")
    return "No response"

In [5]:
def classify_text_gpt(system_prompt, user_prompt, max_retries=5, temperature=0.01):
    retries = 0
    while retries < max_retries:
        try:
            completion = client_gpt.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=temperature,
                response_format={ "type": "json_object" }
            )
            content = completion.choices[0].message.content
            return content
        except Exception as e:
            retries += 1
            wait_time = 2 ** retries
            print(f"Rate limit hit. Retrying in {wait_time} seconds...")
            time.sleep(wait_time)
    print("Max retries exceeded.")
    return "No response"

## gpt

In [6]:
# Load previously saved results if they exist
output_file = 'results_gpt_fewshot.csv'

# Check if the file exists, and if so, load the saved progress
if os.path.exists(output_file):
    processed_df = pd.read_csv(output_file)
    processed_indices = set(processed_df['index'])
    results = processed_df.to_dict(orient='records')
    print(f"Resuming from {len(results)} processed rows.")
else:
    results = []
    processed_indices = set()

# Iterate over the DataFrame and process remaining rows
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    if index in processed_indices:
        # Skip rows that have already been processed
        continue
    
    model_output = classify_text_gpt(row['system_prompt'], row['user_prompt'])

    # Append the result to the list
    results.append({
        'index': index,
        'output': model_output
    })

    # Save results to CSV after each iteration to ensure progress is saved
    pd.DataFrame(results).to_csv(output_file, index=False)

print("Processing complete.")

100%|██████████| 3000/3000 [1:09:41<00:00,  1.39s/it]  

Processing complete.


In [7]:
# Load previously saved results if they exist
output_file = 'results_mistral_fewshot.csv'

# Check if the file exists, and if so, load the saved progress
if os.path.exists(output_file):
    processed_df = pd.read_csv(output_file)
    processed_indices = set(processed_df['index'])
    results = processed_df.to_dict(orient='records')
    print(f"Resuming from {len(results)} processed rows.")
else:
    results = []
    processed_indices = set()

# Iterate over the DataFrame and process remaining rows
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    if index in processed_indices:
        # Skip rows that have already been processed
        continue
    
    model_output = classify_text(row['system_prompt'], row['user_prompt'], model="mistral-large-latest")

    # Append the result to the list
    results.append({
        'index': index,
        'output': model_output
    })

    # Save results to CSV after each iteration to ensure progress is saved
    pd.DataFrame(results).to_csv(output_file, index=False)

print("Processing complete.")


 47%|████▋     | 1418/3000 [35:35<39:06,  1.48s/it]  

Rate limit hit. Retrying in 2 seconds...


 55%|█████▍    | 1640/3000 [46:59<28:59,  1.28s/it]    

Rate limit hit. Retrying in 2 seconds...


100%|██████████| 3000/3000 [1:21:56<00:00,  1.64s/it]

Processing complete.
